# Extraction task

In [1]:
from datasets import load_dataset

full_pii_ds = load_dataset("gretelai/synthetic_pii_finance_multilingual")

README.md:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/48.4M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/5.42M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50346 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5594 [00:00<?, ? examples/s]

In [2]:
full_pii_ds

DatasetDict({
    train: Dataset({
        features: ['level_0', 'index', 'document_type', 'document_description', 'expanded_type', 'expanded_description', 'language', 'language_description', 'domain', 'generated_text', 'pii_spans', 'conformance_score', 'quality_score', 'toxicity_score', 'bias_score', 'groundedness_score'],
        num_rows: 50346
    })
    test: Dataset({
        features: ['level_0', 'index', 'document_type', 'document_description', 'expanded_type', 'expanded_description', 'language', 'language_description', 'domain', 'generated_text', 'pii_spans', 'conformance_score', 'quality_score', 'toxicity_score', 'bias_score', 'groundedness_score'],
        num_rows: 5594
    })
})

In [3]:
full_pii_ds["train"][0]

{'level_0': 40012,
 'index': 40012,
 'document_type': 'Supply Chain Management Agreement',
 'document_description': 'A legal contract outlining the terms and conditions of managing the flow of goods and services within a supply chain, including responsibilities and performance metrics.',
 'expanded_type': 'Vendor Management Contract',
 'expanded_description': 'This subtype involves the contractual agreements and guidelines for managing vendors within the supply chain. It covers aspects such as vendor selection, performance evaluation, and dispute resolution procedures.',
 'language': 'English',
 'language_description': 'English language as spoken in the United States, the UK, or Canada',
 'domain': 'finance',
 'generated_text': 'SUPPLY CHAIN MANAGEMENT AGREEMENT\n\nThis Supply Chain Management Agreement (the "Agreement") is entered into as of this 1st day of March, 2021 (the "Effective Date"), by and between Cameron-Mcknight, a company organized and existing under the laws of the state

In [4]:
good_data = full_pii_ds.filter(lambda x : x["conformance_score"] > 90)

Filter:   0%|          | 0/50346 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5594 [00:00<?, ? examples/s]

In [5]:
good_data

DatasetDict({
    train: Dataset({
        features: ['level_0', 'index', 'document_type', 'document_description', 'expanded_type', 'expanded_description', 'language', 'language_description', 'domain', 'generated_text', 'pii_spans', 'conformance_score', 'quality_score', 'toxicity_score', 'bias_score', 'groundedness_score'],
        num_rows: 9197
    })
    test: Dataset({
        features: ['level_0', 'index', 'document_type', 'document_description', 'expanded_type', 'expanded_description', 'language', 'language_description', 'domain', 'generated_text', 'pii_spans', 'conformance_score', 'quality_score', 'toxicity_score', 'bias_score', 'groundedness_score'],
        num_rows: 1040
    })
})

# Prepare dataset

In [6]:
import json

In [7]:
SYSTEM_PROMPT = "You are an expert at identifying sensitive personal information."

In [8]:
# i converted to uppsercase
PII_LABELS = ['ACCOUNT_PIN', 'API_KEY', 'BANK_ROUTING_NUMBER', 'BBAN', 'COMPANY', 'CREDIT_CARD_NUMBER', 'CREDIT_CARD_SECURITY_CODE', 'CUSTOMER_ID', 'DATE', 'DATE_OF_BIRTH', 'DATE_TIME', 'DRIVER_LICENSE_NUMBER', 'EMAIL', 'EMPLOYEE_ID', 'FIRST_NAME', 'IBAN', 'IPV4', 'IPV6', 'LAST_NAME', 'LOCAL_LATLNG', 'NAME', 'PASSPORT_NUMBER', 'PASSWORD', 'PHONE_NUMBER', 'SSN', 'STREET_ADDRESS', 'SWIFT_BIC_CODE', 'TIME', 'USER_NAME']

In [9]:
def process_example(example):
    curr_close_idx = 0
    
    # iterate over dics of annotations, care account for closing index
    processed_string = []
    
    orig_text = example["generated_text"]
    
    pii_d = json.loads(example["pii_spans"])
    
    for annot_d in pii_d:
        s, e = annot_d["start"], annot_d["end"]
        label = annot_d["label"].upper()
        
        # add string up to this annotation
        if s > curr_close_idx:
            processed_string.append(orig_text[curr_close_idx:s])
        
        # update curr_close_idx
        curr_close_idx = e
        
        # add the <TAG>xxxx</TAG>
        tmp = f"<START OF IDENTIFIABLE INFORMATION : {label}>{orig_text[s:e]}<END OF IDENTIFIABLE INFORMATION : {label}>"
        
        processed_string.append(tmp)
        
    # handle end of sentence
    if curr_close_idx < len(orig_text):
        processed_string.append(orig_text[curr_close_idx:])
    
    pii_sequence_string = ''.join(processed_string)
    
    text_lang = example["language"]
    
    llama3template = f"""Below is a DOCUMENT written in {text_lang} that may contain sensitive identifiable information, such as people's names or telephone numbers.

Find all the sensitive identifiable information in the DOCUMENT and tag these sections of the DOCUMENT.

Use only the tags in the IDENTIFIABLE INFORMATION TAGS list below.

Reply with a LABELLED DOCUMENT which contains all the sensitive identifiable information.

Follow the formatting of the EXAMPLE given below without deviation.

### IDENTIFIABLE INFORMATION TAGS:

- ACCOUNT_PIN
- API_KEY
- BANK_ROUTING_NUMBER
- BBAN
- COMPANY
- CREDIT_CARD_NUMBER
- CREDIT_CARD_SECURITY_CODE
- CUSTOMER_ID
- DATE
- DATE_OF_BIRTH
- DATE_TIME
- DRIVER_LICENSE_NUMBER
- EMAIL
- EMPLOYEE_ID
- FIRST_NAME
- IBAN
- IPV4
- IPV6
- LAST_NAME
- LOCAL_LATLNG
- NAME
- PASSPORT_NUMBER
- PASSWORD
- PHONE_NUMBER
- SSN
- STREET_ADDRESS
- SWIFT_BIC_CODE
- TIME
- USER_NAME

### EXAMPLE:

document: "My name is Bob Smith and you can call me on 0800-134-5813 today."
labelled document: "My name is <START OF IDENTIFIABLE INFORMATION : NAME>Bob Smith<END OF IDENTIFIABLE INFORMATION : NAME> and you can call me on <START OF IDENTIFIABLE INFORMATION : PHONE_NUMBER>0800-134-5813<END OF IDENTIFIABLE INFORMATION : PHONE_NUMBER> today."

### DOCUMENT:

{orig_text}
"""
    
    return llama3template, pii_sequence_string

In [10]:
userp,ans = process_example(good_data["train"][0])

print(userp)

Below is a DOCUMENT written in English that may contain sensitive identifiable information, such as people's names or telephone numbers.

Find all the sensitive identifiable information in the DOCUMENT and tag these sections of the DOCUMENT.

Use only the tags in the IDENTIFIABLE INFORMATION TAGS list below.

Reply with a LABELLED DOCUMENT which contains all the sensitive identifiable information.

Follow the formatting of the EXAMPLE given below without deviation.

### IDENTIFIABLE INFORMATION TAGS:

- ACCOUNT_PIN
- API_KEY
- BANK_ROUTING_NUMBER
- BBAN
- COMPANY
- CREDIT_CARD_NUMBER
- CREDIT_CARD_SECURITY_CODE
- CUSTOMER_ID
- DATE
- DATE_OF_BIRTH
- DATE_TIME
- DRIVER_LICENSE_NUMBER
- EMAIL
- EMPLOYEE_ID
- FIRST_NAME
- IBAN
- IPV4
- IPV6
- LAST_NAME
- LOCAL_LATLNG
- NAME
- PASSPORT_NUMBER
- PASSWORD
- PHONE_NUMBER
- SSN
- STREET_ADDRESS
- SWIFT_BIC_CODE
- TIME
- USER_NAME

### EXAMPLE:

document: "My name is Bob Smith and you can call me on 0800-134-5813 today."
labelled document: "My 

In [11]:
print(ans)

SUPPLY CHAIN RESILIENCE FRAMEWORK

This Supply Chain Resilience Framework (the "Agreement") is entered into as of this <START OF IDENTIFIABLE INFORMATION : DATE>1st day of August, 2021<END OF IDENTIFIABLE INFORMATION : DATE>, by and between <START OF IDENTIFIABLE INFORMATION : NAME>Romeo R. Druso<END OF IDENTIFIABLE INFORMATION : NAME>, residing at <START OF IDENTIFIABLE INFORMATION : STREET_ADDRESS>406 Joshua Square, 08055, North Andrewberg<END OF IDENTIFIABLE INFORMATION : STREET_ADDRESS> ("Client") and <START OF IDENTIFIABLE INFORMATION : BBAN>LGWI52890450971869<END OF IDENTIFIABLE INFORMATION : BBAN> ("Service Provider").

WHEREAS, Client desires to engage Service Provider for the provision of supply chain management services, and Service Provider is willing to provide such services, subject to the terms and conditions set forth herein;

NOW, THEREFORE, in consideration of the mutual covenants contained herein and for other good and valuable consideration, the receipt and sufficien

In [12]:
grpo_train_dataset = []

for example in good_data["train"]:
    user_prompt, gold_answer = process_example(example)
    if user_prompt is not None and gold_answer is not None:
        curr_datapoint = { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': user_prompt}
        ],
        'answer': gold_answer}
        grpo_train_dataset.append(curr_datapoint)

In [13]:
grpo_train_dataset[0]

{'prompt': [{'role': 'system',
   'content': 'You are an expert at identifying sensitive personal information.'},
  {'role': 'user',
   'content': 'Below is a DOCUMENT written in English that may contain sensitive identifiable information, such as people\'s names or telephone numbers.\n\nFind all the sensitive identifiable information in the DOCUMENT and tag these sections of the DOCUMENT.\n\nUse only the tags in the IDENTIFIABLE INFORMATION TAGS list below.\n\nReply with a LABELLED DOCUMENT which contains all the sensitive identifiable information.\n\nFollow the formatting of the EXAMPLE given below without deviation.\n\n### IDENTIFIABLE INFORMATION TAGS:\n\n- ACCOUNT_PIN\n- API_KEY\n- BANK_ROUTING_NUMBER\n- BBAN\n- COMPANY\n- CREDIT_CARD_NUMBER\n- CREDIT_CARD_SECURITY_CODE\n- CUSTOMER_ID\n- DATE\n- DATE_OF_BIRTH\n- DATE_TIME\n- DRIVER_LICENSE_NUMBER\n- EMAIL\n- EMPLOYEE_ID\n- FIRST_NAME\n- IBAN\n- IPV4\n- IPV6\n- LAST_NAME\n- LOCAL_LATLNG\n- NAME\n- PASSPORT_NUMBER\n- PASSWORD\n- PHO

# How to find the reference answer without tags

In [14]:
s = """Bob Smith and you can call me on 0800-134-5813 today."\nlabelled document: "My name is <START OF IDENTIFIABLE INFORMATION : NAME>Bob Smith<END OF IDENTIFIABLE INFORMATION : NAME> and you can call me on <START OF IDENTIFIABLE INFORMATION : PHONE_NUMBER>0800-134-5813<END OF IDENTIFIABLE INFORMATION : PHONE_NUMBER> today."\n\n### DOCUMENT:\n\nSUPPLY CHAIN RESILIENCE FRAMEWORK\n\nThis Supply Chain Resilience Framework (the "Agreement") is entered into as of this 1st day of August, 2021, by and between Romeo R. Druso, residing at 406 Joshua Square, 08055, North Andrewberg ("Client") and LGWI52890450971869 ("Service Provider").\n\nWHEREAS, Client desires t"""

print(s.index("### DOCUMENT:\n\n"))
print(len("### DOCUMENT:\n\n"))

322
15


In [15]:
s[322+15:]

'SUPPLY CHAIN RESILIENCE FRAMEWORK\n\nThis Supply Chain Resilience Framework (the "Agreement") is entered into as of this 1st day of August, 2021, by and between Romeo R. Druso, residing at 406 Joshua Square, 08055, North Andrewberg ("Client") and LGWI52890450971869 ("Service Provider").\n\nWHEREAS, Client desires t'

### Regex for removing brackets

In [16]:
import re

example_text = "My name is <START OF IDENTIFIABLE INFORMATION : NAME>Bob Smith<END OF IDENTIFIABLE INFORMATION : NAME> and you can call me on <START OF IDENTIFIABLE INFORMATION : PHONE_NUMBER>0800-134-5813<END OF IDENTIFIABLE INFORMATION : PHONE_NUMBER> today."

def remove_uppercase_tags(text):
    clean_text = re.sub(r'<[A-Z\s:_]+>', '', text)
    return clean_text

clean_text = remove_uppercase_tags(example_text)
print(clean_text)

My name is Bob Smith and you can call me on 0800-134-5813 today.


# Reward functions

In [17]:
def strict_extractive_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    # raw model predictions
    responses = [completion[0]['content'] for completion in completions]
    # get the original input text i.e. without the tags
    q = prompts[0][-1]['content']
    # get only the gold text document, it begins at ### DOCUMENT:\n\n
    #print(q.index("### DOCUMENT:\n\n"))
    #print(len("### DOCUMENT:\n\n"))
    start_idx = q.index("### DOCUMENT:\n\n")
    original_document = q[start_idx + len("### DOCUMENT:\n\n"):]

    # now process the responses: remove all the <TAGS> from them
    extracted_responses = [remove_uppercase_tags(r) for r in responses]
    
    print('-'*20, f"Original document:\n{original_document}", f"\nRaw response:\n{responses[0]}", f"\nProcessed response:\n{extracted_responses[0]}")
    
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

In [18]:
import pandas as pd
from datasets import Dataset

# convert to HF dataset
df = pd.DataFrame(grpo_train_dataset)
train_data = Dataset.from_pandas(df, split="train")

In [19]:
train_data

Dataset({
    features: ['prompt', 'answer'],
    num_rows: 9197
})

# Now training and Unsloth stuff

In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
!pip install unsloth vllm

# feb 2025
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

!pip install triton==3.1.0
!pip install -U pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 6.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 MB 26.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 82.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.6/343.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━

In [22]:
from unsloth import FastLanguageModel, PatchFastRL

# feb 2025
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [23]:
RANDOM_SEED = 1729


In [24]:
from unsloth import is_bfloat16_supported

import torch

In [25]:
# 12hr training it seems in Kaggle
#MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
#MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct" # want to see if can use a very small model for the task in any case

In [26]:
max_seq_length = 1024
#max_seq_length = 2048 # 12hr training with 7B Qwen
lora_rank = 64

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank, # todo: check if should do 2x or just 1x rank
    use_gradient_checkpointing = "unsloth",
    random_state = RANDOM_SEED,
)

INFO 02-25 22:18:54 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-0.5b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.66%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 6.79 GB. Also swap space = 5 GB.
WARNING 02-25 22:18:57 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 02-25 22:19:09 config.py:549] This model sup

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.2.15 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


In [27]:
!pwd

/kaggle/working


In [28]:
!mkdir outputs

In [29]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 1500,
    max_completion_length = 1500,
    num_train_epochs = 1, # Set to 1 for a full training run
    #max_steps = 250,
    save_steps = 500,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

In [30]:
import time

In [32]:
start_time = time.time()

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        strict_extractive_reward_func,
    ],
    args = training_args,
    train_dataset = train_data,
)

train_results = trainer.train()

end_time = time.time()

print("TIME -> ", end_time-start_time)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,197 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,149
 "-____-"     Number of trainable parameters = 35,192,832
Unsloth: Input IDs of length 1096 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.


-------------------- Original document:
Confirmación de Canje de Cheque de Viajero

Fecha de Canje: 15/06/2023

Estimado(a) Mette van Lucel,

Nos complace confirmar que hemos recibido y canjeado con éxito su cheque de viajero en la sucursal de nuestra institución financiera ubicada en Vicolo Ferdinando, 3 Piano 9. A continuación, encontrará los detalles de la transacción:

IP: 196.52.112.128
Fecha de Canje: 15/06/2023
Hora de Canje: 11:35
Monto Canjeado: €500,00 EUR

Como resultado de este canje, recibirá la cantidad de €500,00 EUR en efectivo. Verifique cuidadosamente el monto canjeado con el cajero antes de finalizar la transacción.

Le agradecemos por elegir nuestra institución financiera para canjear su cheque de viajero. Si tiene alguna pregunta o inquietud, no dude en comunicarse con nosotros al +34-123-456-789 o a [info@bankname.com](mailto:info@bankname.com). Estamos aquí para ayudarlo.

Atentamente,

[Nombre del Banco]
Vicolo Ferdinando, 3 Piano 9
12345 Madrid, España
+34-123-

Unsloth: Input IDs of length 1360 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.


-------------------- Original document:
**BOLETA DE EMBARQUE**

No. de Orden: 22CE-28947-NCC
Fecha: 15 de Marzo de 2022

**DATOS DEL EXPEDIENTE**

Línea Naviera: Compañía Marítima del Noroeste
Buque: M/N "Noruega"
Puerto de Carga: Barcelona, España
Puerto de Descarga: Veracruz, México
Fecha de Salida: 20 de Marzo de 2022
Fecha de Llegada: 10 de Abril de 2022

**DESCRIPCIÓN DE LA CARGA**

Tipo de Carga: Mercancía General
Marcas/Números: -
Piezas: 10
Peso Bruto: 5,000 kg
Volumen: 12 m³
Descripción: Muebles y efectos personales

**DATOS DEL REMITENTE**

Nombre: Nathalie C. Chartier
Dirección: Calle 73 Ängsgränd, Apt. 8
Ciudad: Barcelona
Código Postal: 08001
País: España
Teléfono: +34 654 123 456
Correo Electrónico: [nathaliechartier@email.com](mailto:nathaliechartier@email.com)

**DATOS DEL CONSIGNATARIO**

Nombre: José M. López
Dirección: Calle 5 de Mayo 45, Col. Centro
Ciudad: Veracruz
Código Postal: 91700
País: México
Teléfono: +52 229 123 4567
Correo Electrónico: [jmlopez@email.com.mx

Unsloth: Input IDs of length 1377 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.


-------------------- Original document:
"iban","nombre","direccion","compra"
"FR7723981966468116354427484","Lucy E. Laurent","77 Camino de Javi Fuertes, Apt. 43","150.50"
 
Raw response:
|       |
| ID: Customer Name       | Customer ID                | Name                | IBAN                      | Iban
| --------------------- | ------------------------------ | ------------------------ | ------------------------ | --------------------
|      Orange Juice|
| [<START OF IDENTIFIABLE INFORMATION : BUSINESS_ID>]
| [ Customer Name : Birthday]| Address : Check the conditions and report if necessary.| [ Push Payment ID]|
| [<START OF IDENTIFIABLE INFORMATION :贩卖商品>]
| <END OF IDENTIFIABLE INFORMATION :贩卖商品>|
|     /ldf/"aa4rnboq"|
| <END OF IDENTIFIABLE INFORMATION : アリーテム>|
|   /ldf/"
|      | <END OF IDENTIFIABLE INFORMATION:  | [ Pay By Card ID]
|     /ldf/"p8jb6tzq"|
|    /ldf/"
|      | [ Signatory ID,, ]|
|  /ldf/"551nv3j8"|
|      | [<END OF IDENTIFIABLE INFORMATION: 箱号>]|
|  /ldf/

Unsloth: Input IDs of length 1029 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.


-------------------- Original document:
**Reseförfrågan: Resa**

-------------------------------------------------

**Ansökareinformation**

Namn: Clara José Mari Gallo
Personnummer: T-992863-S
Kund-ID: K-123456789

**Kontaktinformation**

E-postadress: [clara.gallo@exempel.se](mailto:clara.gallo@exempel.se)
Telefonnummer: 070-123 45 67

**Adress**

Gatuadress: 70 Piazza Giada, 25087, Celletta
Postnummer: 25087
Postort: Celletta
Land: Italien

**Resebeskednad**

Resmål: Paris, Frankrike
Resedatum: 15 augusti 2023 - 30 augusti 2023 (16 dagar)
Antal resenärer: 1 person

**Resebudget**

Flygresor: 4 500 SEK
Hotell: 6 000 SEK
Resmålsrelaterade kostnader: 3 500 SEK
Total resbudget: 14 000 SEK

**Inkomst och arbete**

Arbetsplats: Exempelbolaget AB
Sysselsättning: Webbutvecklare
Månadslön: 30 000 SEK

**Bankinformation**

Bank: Swedbank
Kontonummer: 1234 5678 90-1
IP-adress: 4.5.82.174

**Övrigt**

Är du försäkrad under resan? Ja
Har du några speciella behov eller önskemål under resan? Nej



Step,Training Loss,reward,reward_std,completion_length,kl,rewards / strict_extractive_reward_func


KeyboardInterrupt: 

# Interrupted

- seems dataset is all over the place
- also not clear what the reference document is looking at here, need to be clear that it is looking at the base document ?

In [ ]:
SAVE_NAME = "Qwen2.5-0.5B-Instruct"

# save LoRA adapters here
model.push_to_hub_merged(f"benjaminzwhite/{SAVE_NAME}_TextFocus-GRPO_LoRA-adapters", tokenizer, save_method = "lora", token = "")